In [2]:
import numpy as np
import pandas as pd
event = pd.read_csv("event.csv")

train = pd.read_csv("click_train.csv")

train.head()


,displayId,adId,clicked
0,1210227,7182,0
1,1210227,7125,0
2,1210227,7181,0
3,1210227,535,0
4,1210227,7174,1


Merging user_page_view

In [18]:
userPageView = pd.read_csv("user_page_view.csv")
userPageView

,userId,pageId,timestamp
0,821961,8116,1579599211445
1,15321,9533442,1579599211443
2,1125090,9410379,1579599211440
3,407101,8616213,1579599211429
4,781615,9543366,1579599211429
...,...,...,...
67229816,2424805,5307668,1578429000208
67229817,1474341,6245932,1578429000148
67229818,1796890,6245082,1578429000091
67229819,855919,6246403,1578429000033


In [8]:
eventFeatures = ['displayId', 'pageId', 'widgetId', 'userId']
cleanEvent = event[eventFeatures]
cleanEvent.head()

,displayId,pageId,widgetId,userId
0,4706262,3543873,6262,2688642
1,4706267,6245475,607,2688641
2,4706260,4416499,11458,2688638
3,4706255,6246028,9358,1962852
4,4706256,5327047,9358,2687719


In [6]:
string = 'displayId'
displayIdForEvent = event[string]
displayIdForEvent

0          4706262
1          4706267
2          4706260
3          4706255
4          4706256
            ...   
1083066     243935
1083067     243775
1083068     243745
1083069     243931
1083070     243751
Name: displayId, Length: 1083071, dtype: int64

## With features that really cares

In [74]:
expandedXTrain = X_train.copy(deep=True)
expandedXTrain.head()
event = pd.read_csv('event.csv')

In [81]:
type(expandedXTrain)

pandas.core.frame.DataFrame

add userId to expandedXTrain {

In [83]:
userIdList = []
for displayId in expandedXTrain.iloc[:, 0]:
    userIdList.append((event.loc[event['displayId'] == displayId])['userId'])
print(userIdList[:10])

KeyboardInterrupt: 

In [73]:
click_test = pd.read_csv('click_test.csv')
click_test.head(100)

,displayId,adId
0,151650,7585
1,151650,6257
2,151650,6690
3,151938,7454
4,151938,7370
...,...,...
95,237789,7091
96,237789,5965
97,237789,5205
98,240394,7715


# Get Accuracy

## Seyed Merging

In [69]:
eventData = pd.read_csv('event.csv')
clickTrainData = pd.read_csv('click_train.csv')

In [75]:
eventData2=eventData.drop(columns=['timestamp', 'pageId', 'OS', 'browser', 'device', 'widgetId'])
ev=eventData2.sort_values(by=['displayId']).to_numpy()
ctrain=clickTrainData.sort_values(by=['displayId']).to_numpy()

result=np.zeros((ctrain.shape[0],ctrain.shape[1]+ev.shape[1]-1),dtype=np.int64)

(n,m)=result.shape
(evn,evm)=ev.shape
curr=0
for i in range(n):
    while curr<evn and ev[curr][0]<ctrain[i][0]:
        curr+=1
    if curr<evn and ctrain[i][0]==ev[curr][0]:
        cnt=0
        for j in range(0,ctrain.shape[1]):
            result[i][cnt]=ctrain[i][j]
            cnt+=1
        for j in range(1,evm):
            result[i][cnt]=ev[curr][j]
            cnt+=1
    else:
        print('One missed row')
bigData = pd.DataFrame(data=result,columns=['displayId', 'adId','clicked','userId'])
bigData.describe()

,displayId,adId,clicked,userId
count,3.642244e+06,3.642244e+06,3.642244e+06,3.642244e+06
mean,3.105835e+06,3.853480e+03,2.230227e-01,1.530336e+06
std,9.807950e+05,2.465609e+03,4.162735e-01,7.075787e+05
min,1.210227e+06,6.000000e+00,0.000000e+00,8.000000e+00
25%,2.246184e+06,6.810000e+02,0.000000e+00,9.852190e+05
50%,3.162424e+06,5.049000e+03,0.000000e+00,1.578216e+06
75%,3.913442e+06,5.890000e+03,0.000000e+00,2.124848e+06
max,4.706267e+06,7.182000e+03,1.000000e+00,2.688642e+06


In [76]:
bigData.head()

,displayId,adId,clicked,userId
0,1210227,7182,0,782749
1,1210227,7125,0,782749
2,1210227,7181,0,782749
3,1210227,535,0,782749
4,1210227,7174,1,782749


## merge userId with wordId

In [77]:
adTitle = pd.read_csv('ad_title.csv')

In [83]:
bigData.columns

Index(['displayId', 'adId', 'clicked', 'userId'], dtype='object')

In [84]:
ad = adTitle.sort_values(by=['adId']).to_numpy()
newbigData = bigData.sort_values(by=['adId']).to_numpy()

result=np.zeros((newbigData.shape[0],newbigData.shape[1]+ad.shape[1]-1),dtype=np.int64)

(n,m)=result.shape
(evn,evm)=ev.shape
curr=0
for i in range(n):
    while curr<evn and ad[curr][0]<newbigData[i][1]:
        curr+=1
    if curr<evn and newbigData[i][1]==ad[curr][0]:
        cnt=0
        for j in range(0,newbigData.shape[1]):
            result[i][cnt]=newbigData[i][j]
            cnt+=1
        for j in range(1,evm):
            result[i][cnt]=ad[curr][j]
            cnt+=1
    else:
        #tof!
        cnt=0
        for j in range(0,newbigData.shape[1]):
            result[i][cnt]=newbigData[i][j]
            cnt+=1
        for j in range(1,evm):
            result[i][cnt]=0
            cnt+=1
newbigData = pd.DataFrame(data=result,columns=['displayId', 'adId', 'clicked','userId', 'wordId'])
newbigData.head()

,displayId,adId,clicked,userId,wordId
0,3162423,6,0,1874592,24
1,3641329,6,0,2133247,24
2,3827885,6,0,1102761,24
3,3827884,6,1,2230075,24
4,3827869,6,1,2230089,24


In [85]:
sortedBigData = newbigData.sort_values(by=['displayId'])

In [86]:
sortedBigData.head()

,displayId,adId,clicked,userId,wordId
3642243,1210227,7182,0,782749,108
3639160,1210227,7181,0,782749,853
745066,1210227,535,0,782749,13
3625504,1210227,7125,0,782749,2544
3636967,1210227,7174,1,782749,29


## Spliting train data

67% Training data, 33% Test set

In [169]:
train = sortedBigData.copy(deep = True)
train.head()

,displayId,adId,clicked,userId,wordId
3642243,1210227,7182,0,782749,108
3639160,1210227,7181,0,782749,853
745066,1210227,535,0,782749,13
3625504,1210227,7125,0,782749,2544
3636967,1210227,7174,1,782749,29


In [170]:
train.shape
train = train.to_numpy()

In [171]:
train.shape

(3642244, 5)

In [180]:
i = 0
y_train = []
X_train = []
y_test = []
X_test = []
samples = 0
while(i < train.shape[0]):
    k = i + 1
    # here 0 is the column that is displayId
    while(k < train.shape[0] and train[i][0] == train[k][0]):
        k += 1
    # here 2 is clicked column
    # :2
    if(np.random.randint(low = 1, high = 100) < 33):
        for j in range(i, k):
            y_test.append(train[j][2])
            
            X_test.append([*train[j][:2], *train[j][3:]])
    else:
        for j in range(i, k):
            y_train.append(train[j][2])
            X_train.append([*train[j][:2], *train[j][3:]])
    i = k
    samples += 1
print(samples)

812303


In [181]:
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

2464587
2464587
1177657
1177657


In [182]:
for i in range(10):
    print(X_train[i], y_train[i])
print("mashtigholam")
for i in range(10):
    print(X_test[i], y_test[i])

[1210227, 7182, 782749, 108] 0
[1210227, 7181, 782749, 853] 0
[1210227, 535, 782749, 13] 0
[1210227, 7125, 782749, 2544] 0
[1210227, 7174, 782749, 29] 1
[1214987, 5652, 786509, 1] 0
[1214987, 7125, 786509, 2544] 0
[1214987, 7092, 786509, 45] 1
[1248098, 7182, 791563, 108] 0
[1248098, 17, 791563, 31] 1
mashtigholam
[1271266, 7173, 792543, 2454] 0
[1271266, 7174, 792543, 29] 0
[1271266, 6693, 792543, 31] 0
[1271266, 7181, 792543, 853] 0
[1271266, 7182, 792543, 108] 1
[1292338, 7113, 796194, 29] 0
[1292338, 7092, 796194, 45] 0
[1292338, 6703, 796194, 544] 1
[1292338, 7167, 796194, 506] 0
[1299669, 28, 810968, 116] 0


In [183]:
X_train = pd.DataFrame(data=X_train,columns=['displayId', 'adId', 'userId', 'wordId'])

In [184]:
X_test = pd.DataFrame(data=X_test,columns=['displayId', 'adId', 'userId', 'wordId'])
y_train = pd.DataFrame(data=y_train,columns=['clicked'])
y_test = pd.DataFrame(data=y_test,columns=['clicked'])

Done

# DecisionTreeRegressor

In [185]:
X_trainWithoutDis = X_train.drop(columns = ['displayId', 'adId'])
X_testWithoutDis = X_test.drop(columns = ['displayId', 'adId'])

## without any features

In [214]:
from sklearn.tree import DecisionTreeClassifier
treeModel = DecisionTreeClassifier(random_state=1, max_depth=20)
treeModel.fit(X_trainWithoutDis, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=20, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [215]:
X_trainWithoutDis.head()

,userId,wordId
0,782749,108
1,782749,853
2,782749,13
3,782749,2544
4,782749,29


In [216]:
y_train.head()

,clicked
0,0
1,0
2,0
3,0
4,1


In [217]:
X_testWithoutDis.head()

,userId,wordId
0,792543,2454
1,792543,29
2,792543,31
3,792543,853
4,792543,108


In [218]:
y_test.head()

,clicked
0,0
1,0
2,0
3,0
4,1


In [219]:
predictions = treeModel.predict_proba(X_testWithoutDis)
type(predictions)

numpy.ndarray

In [220]:
for i in range(1000):
    print(predictions[i])

[1. 0.]
[0.80978261 0.19021739]
[0.64791529 0.35208471]
[0.75395431 0.24604569]
[0.67874166 0.32125834]
[0.75 0.25]
[0.6743295 0.3256705]
[0.79084874 0.20915126]
[0.86553323 0.13446677]
[0.67295189 0.32704811]
[0.8363747 0.1636253]
[1. 0.]
[0.72929199 0.27070801]
[0.72929199 0.27070801]
[0.70658683 0.29341317]
[0.81223329 0.18776671]
[0.73942094 0.26057906]
[0.73942094 0.26057906]
[0.86005089 0.13994911]
[0.78387429 0.21612571]
[1. 0.]
[0.73855964 0.26144036]
[1. 0.]
[0.79411765 0.20588235]
[0.75389902 0.24610098]
[0.90243902 0.09756098]
[0.87831858 0.12168142]
[0.79812207 0.20187793]
[0.85512795 0.14487205]
[0.92413793 0.07586207]
[0.90243902 0.09756098]
[0.85199456 0.14800544]
[1. 0.]
[0.80259683 0.19740317]
[0.86005089 0.13994911]
[0.85199456 0.14800544]
[0.86005089 0.13994911]
[0.90243902 0.09756098]
[1. 0.]
[0.80259683 0.19740317]
[0.92413793 0.07586207]
[1. 0.]
[0.83145706 0.16854294]
[0.80182232 0.19817768]
[0.81640525 0.18359475]
[0.79812207 0.20187793]
[0.75389902 0.24610098]


[0.64791529 0.35208471]
[0.72773637 0.27226363]
[0.66782007 0.33217993]
[0.76808213 0.23191787]
[0.66782007 0.33217993]
[0.72773637 0.27226363]
[0.72773637 0.27226363]
[0.72246624 0.27753376]
[0.86544343 0.13455657]
[0.72773637 0.27226363]
[1. 0.]
[0.76808213 0.23191787]
[0.72773637 0.27226363]
[0.79831933 0.20168067]
[0.64791529 0.35208471]
[0.86567164 0.13432836]
[0.79084874 0.20915126]
[0.85326087 0.14673913]
[1. 0.]
[0.72773637 0.27226363]
[0.6637931 0.3362069]
[0.73855964 0.26144036]
[1. 0.]
[1. 0.]
[0.84057971 0.15942029]
[0.54935622 0.45064378]
[0.76808213 0.23191787]
[0.76808213 0.23191787]
[0.64791529 0.35208471]
[0.72773637 0.27226363]
[0.64791529 0.35208471]
[0.85326087 0.14673913]
[0.84057971 0.15942029]
[0.73855964 0.26144036]
[0.72773637 0.27226363]
[0.76808213 0.23191787]
[0.73855964 0.26144036]
[0.91549296 0.08450704]
[0.72773637 0.27226363]
[0.66782007 0.33217993]
[0.80786026 0.19213974]
[0.65418895 0.34581105]
[0.76808213 0.23191787]
[0.69230769 0.30769231]
[0.7539543

In [221]:
X_testnp = X_test.to_numpy()
y_testnp = y_test.to_numpy()
correctAns = 0
All = 0
i = 0
while i < X_testnp.shape[0]:
    k = i + 1
    maxId = i
    while k < X_testnp.shape[0] and X_testnp[i][0] == X_testnp[k][0]:
        if predictions[k][1] > predictions[maxId][1]:
            maxId = k
        k += 1
    if y_testnp[maxId] == 1:
        correctAns += 1
    All += 1
    i = k
print(correctAns / All)

0.3394212713442853


In [123]:
type(predictions)

numpy.ndarray

In [54]:
type(y_test)

pandas.core.series.Series

In [57]:
correct = 0
for i in range(predictions.size):
    if 0 == y_test.iloc[i]:
        correct += 1
print((float)(correct) / predictions.size)

0.776649602601126


In [58]:
correct = 0
for i in range(predictions.size):
    if predictions[i] == y_test.iloc[i]:
        correct += 1
print((float)(correct) / predictions.size)

0.6737876484785859


# copy of merge of seyed

In [223]:
click_test = pd.read_csv('click_test.csv')
click_test = click_test.to_numpy()
for i in range(click_test.shape[0] - 1):
    if(click_test[i][0] > click_test[i + 1][0]):
        print(i, click_test[i], click_test[i + 1])

In [ ]:
eventData2=eventData.drop(columns=['OS', 'browser','device','widgetId'])
eventData2.describe()
ev=eventData2.sort_values(by=['displayId']).to_numpy()
ctrain=clickTrainData.sort_values(by=['displayId']).to_numpy()
ctrain.shape
result=np.zeros((ctrain.shape[0],ctrain.shape[1]+ev.shape[1]-2),dtype=np.int64)

(n,m)=result.shape
(evn,evm)=ev.shape
curr=0
for i in range(n):
   while curr<evn and ev[curr][0]<ctrain[i][0]:
       curr+=1
   if curr<evn and ctrain[i][0]==ev[curr][0]:
       cnt=0
       for j in range(1,ctrain.shape[1]):
           result[i][cnt]=ctrain[i][j]
           cnt+=1
       for j in range(1,evm):
           result[i][cnt]=ev[curr][j]
           cnt+=1
   else:
       print('One missed row')
bigData = pd.DataFrame(data=result,columns=['adId','clicked','timestamp','pageId','userId'])
bigData.describe()